In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import TaggedDocument
import string

nltk.download('punkt')
nltk.download('stopwords')

# Cargar el conjunto de datos
file_path = 'newsCorpora-trimmed.csv'
data = pd.read_csv(file_path)

data.columns = ['category', 'text']

# Filtrar solo las categorías de interés
categories = ['b', 't', 'e', 'm']  # Business, Science and Technology, Entertainment, Health
data = data[data['category'].isin(categories)]

# Preprocesamiento de texto
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

data['tokens'] = data['text'].apply(preprocess_text)

# Crear documentos etiquetados
tagged_data = [TaggedDocument(words=row['tokens'], tags=[row['category']]) for index, row in data.iterrows()]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from gensim.models import Doc2Vec

print(tagged_data)

# Entrenar el modelo Doc2Vec
model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=5, workers=4, epochs=20)

# Guardar el modelo
model.save("doc2vec_model")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Convertir los documentos etiquetados a vectores
def vectorize_doc(doc):
    return model.infer_vector(doc.words)

data['vector'] = data['tokens'].apply(lambda x: vectorize_doc(TaggedDocument(x, [0])))

# Crear el conjunto de entrenamiento y prueba
X = list(data['vector'])
y = data['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el clasificador SVM
classifier = SVC(kernel='linear')
classifier.fit(X_train, y_train)

# Evaluar el clasificador
y_pred = classifier.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


In [ ]:
def classify_new_document(text):
    tokens = preprocess_text(text)
    vector = vectorize_doc(TaggedDocument(tokens, [0]))
    return classifier.predict([vector])[0]

# Ejemplo de clasificación
new_document = "New breakthrough in cancer research"
predicted_category = classify_new_document(new_document)
print(f"Predicted category: {predicted_category}")


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

# Paso 1: Normalizar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Paso 2: Balancear el dataset
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

# Paso 3: Definir el árbol de decisión y los parámetros para GridSearchCV
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_balanced, y_train_balanced)

# Paso 4: Evaluar el modelo con los mejores parámetros encontrados
best_classifier = grid_search.best_estimator_
y_pred = best_classifier.predict(X_test_scaled)

# Paso 5: Imprimir la precisión y los mejores parámetros
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")
